# Supplementary Material to: 
# Blueprint for Next Generation Cyber-Physical Resilience using Defense Quantum Machine Learning

# Two-train Example

# Authors


<a href="https://carleton.ca/scs/people/michel-barbeau/">Michel Barbeau</a>

<a href="http://www-public.imtbs-tsp.eu/~garcia_a/web/">Joaquin Garcia-Alfaro</a>

Version: April 7, 2020

We reused code from: <a href="https://pennylane.readthedocs.io/en/stable/introduction/measurements.html">Measurements</a>



## Background on Reinforcement Learning

At the core of Reinforcement Learning is a state-transition model. Triggered by an action, every transition is from one state to another. Associated with each action, there is a numerical reward.
Reinforcement Learning aims to find an action-selection policy that maximizes rewards resulting from a sequence of transitions.

The learning entity is called an agent. It has a state set $S=\{ s_0, s_1,\ldots,s_{n-1} \}$ and a set of actions $A=\{ a_0, a_1,\ldots,a_{m-1} \}$. The reward function $r$ has domain $S \times A$ and co-domain $\mathbb{R}$.

Multiple actions might be available in a given state. At any given time, the policy tells the agent which one to take.

The objective of Reinforcement Learning is finding a policy maximizing the return. A policy is a function $P$ with domain $S$ and co-domain $A$. The goal of Reinforcement Learning is to find a good policy w.r.t. a reward function. That is, the goal is obtain the best possible reward. There are several strategies for obtaining such a policy. The greedy strategy aims to maximize the immediate reward, while ignoring the long term reward. The random strategy picks a random action. The greedy and random are comparison references.

Consideration of long-term reward is captured by a utility function $Q$, with domain $S \times A$ and co-domain $\mathbb{R}$.
Given a state $s$ and an action $a$, $Q(s,a)$ returns the utility associated with performing action $a$ in state $s$, taking into account what is going to occur after $a$ is taken.
The utility function can be recursively defined as:
$$ Q(s,a) = r(s,a) + \gamma \max Q(s',a') $$
$s'$ and $a'$ represent the next state-action pair.
Constant $\gamma$, in $[0,1]$ is a discounting factor, weighting the long-term  reward versus the short term one.
The Reinforcement Learning problem can then be formulated as
$$P(s) = \arg\max_a Q(s, a).$$
This reads as the policy is defined such that in a state $s$ the chosen action maximizes the utility $Q(s, a)$.

## Preamble

In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
import math
import time

## Markov Decision Process (MDP)

For background explantions on this example, see Section IV of "Blueprint for Next Generation Cyber-Physical Resilience using Defense Quantum Machine Learning".

Create the MDP environment.

In [2]:
n, m = 3, 2; # number of states, number of actions
take_loop = 0
take_bypass = 1
# init transition matrix
T = np.zeros((n, n, m),dtype=np.int8)
T[0][0][take_loop] = 0.5
T[0][0][take_bypass] = 0.5
T[0][1][take_loop] = 0.5
T[0][1][take_bypass] = 0.5
# init reward matrix
R = np.zeros((n, n, m),dtype=np.int8)
R[0][0][take_loop] = 0
R[0][0][take_bypass] = 4
R[0][1][take_loop] = 0
R[0][1][take_bypass] = 2
# Terminal states
TerminalStates = [1, 2]
# generate binary array representation of states
states = np.array([[0,0],[0,1],[1,0],[1,1]])
#print(states)
# print(R)

## Quantum Reinforcement Learning

Creation of a quantum device.

In [3]:
# Number of qubits: This number determines the size of the problem, in single data
# items, that can be handled by the program.
num_qubits = 2

In [4]:
dev = qml.device('default.qubit', wires=num_qubits)

Create the circuit for the variational classifier.

In [5]:
def layer(W):
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 0])

### Encoding

The state encoding used for this example is $s \rightarrow \vert s \rangle$.

In [6]:
def statepreparation(x):
    # map a list of zeros and ones to a quantum state
    qml.BasisState(x, wires=[0, 1])

Definition of the quantum node, i.e., the structure of the variational circuit. Formal parameter "weights" is the number of repetitions of the variational circuit layer structure. Formal parameter "x" is an input quantum state. 

In [33]:
@qml.qnode(dev)
def circuit(weights, x=None):
    statepreparation(x)
    for W in weights:
        layer(W)
    # return expected values 
    # return [qml.expval.PauliZ(0),qml.expval.PauliZ(1)]
    # return [qml.expval.PauliZ(i) for i in range(2)]
    # return qml.probs(0)
    return qml.expval.PauliZ(0)

Actualize the variational circuit.

In [8]:
def W(var, x=None):
    weights = var[0]
    bias = var[1]
    return circuit(weights, x=x) + bias

Cost model.

In [9]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

def cost(var, X, Y):
    predictions = [W(var, x=x) for x in X]
    return square_loss(Y, predictions)

### Reinforcement Learning

In [10]:
X = states
Y = np.zeros(len(X))
print("Initial target output:", (Y+1)/2 )

Initial target output: [0.5 0.5 0.5 0.5]


In [11]:
# initialize the parameter of the variational circuit to 
# random values
np.random.seed(0)
num_layers = 3
# create a "num_layers" by "num_qubits" by three, 3D matrix
var_init = (0.01 * np.random.randn(num_layers, num_qubits, 3), 0.0)
print(var_init)

(array([[[ 0.01764052,  0.00400157,  0.00978738],
        [ 0.02240893,  0.01867558, -0.00977278]],

       [[ 0.00950088, -0.00151357, -0.00103219],
        [ 0.00410599,  0.00144044,  0.01454274]],

       [[ 0.00761038,  0.00121675,  0.00443863],
        [ 0.00333674,  0.01494079, -0.00205158]]]), 0.0)


 ### Step-by-step update of the weights

In [12]:
opt = NesterovMomentumOptimizer(0.5)
batch_size = 5

In [36]:
# probabilties
p = 0.5
q = 0.5
# init reward matrix
R = np.zeros((m, m),dtype=np.int8)
R[take_loop][0] = 0
R[take_loop][1] = 4
R[take_bypass][0] = 0
R[take_bypass][1]= 2
var = var_init
# Expected choice
E = 0 
alpha = 0.5
for i in range(10):
    # choices made at point zero
    # generate a random agent action
    a = np.random.randint(0, 2)
    # generate at random the corresponding reward
    if a==take_loop:
        r = np.random.binomial(1, p, 1)
    else:
        r = np.random.binomial(1, q, 1) 
    reward = R[a][r[0]]   
    print("Agent choice: ", a, " Reward: ", reward)
    # V = (1-alpha)*V + 

    oldY = [E*2-1]
    print(oldY)
    # update variational circuit
    for j in range(10):
        var = opt.step(lambda v: cost(v, [X[0]],oldY), var)
        # update output, according to estimated reward
        newY = [W(var, X[0])]
        acc = accuracy(oldY, newY)
        print("Iter: {:5d}| Accuracy: {:0.7f} ".format((i*10)+j+1, acc))
        if abs(1 - acc) < 0.1:
            break;



Agent choice:  0  Reward:  0
[-1]
Iter:     1| Accuracy: 0.0000000 
Iter:     2| Accuracy: 0.0000000 
Iter:     3| Accuracy: 0.0000000 
Iter:     4| Accuracy: 0.0000000 
Iter:     5| Accuracy: 0.0000000 
Iter:     6| Accuracy: 0.0000000 
Iter:     7| Accuracy: 0.0000000 
Iter:     8| Accuracy: 0.0000000 
Iter:     9| Accuracy: 0.0000000 
Iter:    10| Accuracy: 0.0000000 
Agent choice:  1  Reward:  2
[-1]
Iter:    11| Accuracy: 0.0000000 
Iter:    12| Accuracy: 0.0000000 
Iter:    13| Accuracy: 0.0000000 
Iter:    14| Accuracy: 0.0000000 
Iter:    15| Accuracy: 0.0000000 
Iter:    16| Accuracy: 0.0000000 
Iter:    17| Accuracy: 0.0000000 
Iter:    18| Accuracy: 0.0000000 
Iter:    19| Accuracy: 0.0000000 
Iter:    20| Accuracy: 0.0000000 
Agent choice:  1  Reward:  2
[-1]
Iter:    21| Accuracy: 0.0000000 
Iter:    22| Accuracy: 0.0000000 
Iter:    23| Accuracy: 0.0000000 
Iter:    24| Accuracy: 0.0000000 
Iter:    25| Accuracy: 0.0000000 
Iter:    26| Accuracy: 0.0000000 
Iter:    27| A